In [1]:
import re
import string
import swifter
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Doc

import time

nlp = spacy.load('en_core_web_sm')
import pickle
import random

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')



In [ ]:
df = pd.DataFrame()

with open('data/ingredients_spacied.pickle', 'rb') as f:
    df = pickle.load(f)

df['ingredients'] = df['ingredients'].swifter.apply(lambda x: DocBin().from_bytes(x))

In [ ]:
with open('data/ingredients_spacied_docbins.pickle', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
with open('data/ingredients_spacied_docbins.pickle', 'rb') as f:
    df = pickle.load(f)
    
df

In [ ]:
df['ingredients'] = df['ingredients'].swifter.apply(lambda x: list(x.get_docs(nlp.vocab)))

In [ ]:
df

In [ ]:
with open('data/ingredients_spacied_docs.pickle', 'wb') as f:
    pickle.dump(df, f)

In [2]:
df = pd.read_json('data/recipe1M_layers/layer1.json')[['ingredients']]

In [3]:
def preprocessor(text):
    
    text = re.sub('(\((?:\(??[^\(]*?\)))','',text)
        
    text = text.split(',')[0]
    text = text.split(' or ')[0]
    text = text.split(' for ')[0]
    text = re.sub('[^a-zA-Z]\sto\s[^a-zA-Z]','',text)
    text = text.split(' to ')[0]

    text = text.split(' and ')[0]


    text = text.lower()
    
    # Removes quotation marks.
    text = text.replace('"', "")
    
    # Remove numeric characters.
    text = re.sub('\w*\d\w*', ' ', text)
    
    # Remove puncuation.
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    
    text = re.sub('\s+', ' ', text).strip()
          
    text = " ".join([stemmer.stem(x) for x in text.split(' ') if (x not in unit_words)])

    text = text.title()
    
    combined = text.replace(' ','')
    
#     text = " ".join([combined,text])
    text = combined
    
    return text

unit_words = {
    'all',
     'bunch','bunched'
     'bushel','bushels',
     'c',
     'chopped',
     'clove','cloves',
     'coarse','coarsely',
     'crumbled',
     'cup','cups',
     'dash','dashes',
     'diced',
     'drop','drops',
     'extra',
     'finely',
     'fl. oz',
     'fresh','freshly',
     'g',
     'gallon','gallons',
     'glass','glasses',
     'gram','grams',
     'grated',
     'italian',
     'kg','kgs',
     'lb','lbs',
     'liter','liters',
     'lrg','large',
     'mashed',
     'med','medium',
     'minced',
     'ml','mls',
     'ounce','ounces','oz',
     'package','packaged','pkg',
     'pasta','pastas',
     'pinch','pinches',
     'pint','pints',
     'pound','pounds',
     'purpose',
     'qt',
     'quart','quarts',
     'scoop','scoops',
     'shot','shots',
     'shredded',
     'sifted',
     'sliced','slivered'
     'sm','small',
     'stick','sticks',
     'tablespoon','tablespoons','tbs','tbsp','tbsps',
     'teaspoon','teaspoons','tsp','tsps',
     'thin','thinly',
     'to',
     'virgin',
     'Virgin',
     'white','whites',
     'whole',
     'x',
     'yolk','yolks'
}

In [4]:
# df['instructions'] = df['instructions'].swifter.apply(lambda x: [y['text'] for y in x])
df['ingredients'] = df['ingredients'].swifter.apply(lambda x: [y['text'] for y in x])

Pandas Apply:   0%|          | 0/1029720 [00:00<?, ?it/s]

In [5]:
start_time = time.time()

df['ingredients'] = df['ingredients'].swifter.apply(lambda x: ([preprocessor(y) for y in x]))

print("--- %s seconds ---" % (time.time() - start_time))

Pandas Apply:   0%|          | 0/1029720 [00:00<?, ?it/s]

--- 495.46034574508667 seconds ---


In [6]:
df['ingredients'] = df['ingredients'].swifter.apply(lambda x: ' '.join(x))

Pandas Apply:   0%|          | 0/1029720 [00:00<?, ?it/s]

In [16]:
df['ingredients'] = df['ingredients'].swifter.apply(lambda x: ' '.join(list(set(x.split(' ')))) )

Pandas Apply:   0%|          | 0/1029720 [00:00<?, ?it/s]

In [35]:
from wordcloud import WordCloud
from PIL import Image

lemon_mask = np.array(Image.open("lemon_mask.jpg"))
wc_text = ' '.join(df['ingredients'].iloc[:100000].tolist())
wc = WordCloud(background_color='white', max_words=5000, mask=lemon_mask)
wc.generate(wc_text)
wc.to_file('test.png')

In [18]:
with open('data/ingredients_cleaned.pickle', 'wb') as f:
    pickle.dump(df, f) 

In [19]:
def flatten(t):
    return [item for sublist in t for item in sublist]
from collections import Counter

counted = Counter(flatten(df['ingredients'].swifter.apply(lambda x: x.split(' ')).tolist()))
counted.most_common()

Pandas Apply:   0%|          | 0/1029720 [00:00<?, ?it/s]

[('Salt', 536072),
 ('Sugar', 349522),
 ('Oil', 347623),
 ('Pepper', 341774),
 ('Onion', 327338),
 ('Butter', 304227),
 ('Garlic', 283960),
 ('Egg', 275071),
 ('Flour', 247039),
 ('Chees', 238970),
 ('Ground', 214511),
 ('Oliv', 206898),
 ('Water', 198043),
 ('Cream', 195114),
 ('Powder', 192202),
 ('OlivOil', 182226),
 ('Milk', 180478),
 ('Can', 171074),
 ('Juic', 161931),
 ('Red', 160156),
 ('Dri', 158921),
 ('Sauc', 158790),
 ('Tomato', 157068),
 ('Black', 155139),
 ('Chicken', 144962),
 ('Lemon', 139899),
 ('Vanilla', 137907),
 ('Green', 129187),
 ('Bake', 120250),
 ('Brown', 110369),
 ('Vinegar', 98554),
 ('Veget', 92347),
 ('Parsley', 89647),
 ('Cinnamon', 88120),
 ('Of', 87693),
 ('Leav', 81245),
 ('Wine', 78877),
 ('LemonJuic', 77657),
 ('Extract', 76651),
 ('Cook', 76633),
 ('Potato', 74401),
 ('Slice', 71113),
 ('Beef', 68723),
 ('Chocol', 68387),
 ('Unsalt', 67662),
 ('BakePowder', 67478),
 ('Carrot', 66524),
 ('Mix', 65764),
 ('Soda', 64857),
 ('Rice', 64613),
 ('Celeri', 6

In [ ]:
# df.to_csv('ingredients_cleaned.csv')

In [11]:
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(lowercase=False,min_df=2,max_df=0.8)
svd_model = NMF(100)

doc_word = tfidf.fit_transform(df['ingredients'])
doc_topic = svd_model.fit_transform(doc_word)
doc_word.shape

(1029720, 111208)

In [12]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
display_topics(svd_model, tfidf.get_feature_names(), 5)


Topic  0
Flour, BakePwdr, IceWater, VegetShorten, ColdUnsaltButter

Topic  1
OlivOil, RedWineVinegar, BasilLeav, PineNut, FlatLeafParsley

Topic  2
Sugar, CreamOfTartar, VanillaBean, CakeFlour, Cranberri

Topic  3
Water, Rice, BreadFlour, Yeast, LongGrainRice

Topic  4
Butter, Cream, IceSugar, EvaporMilk, PlainFlour

Topic  5
BrownSugar, RollOat, Oatmeal, Oat, SweetPotato

Topic  6
VanillaExtract, UnsweetenCocoaPowder, CocoaPowder, SemiSweetChocolChip, HeaviWhipCream

Topic  7
SourCream, Salsa, MontereyJackChees, PackagCreamChees, CanCreamOfChickenSoup

Topic  8
Milk, SharpCheddarChees, SelfRiseFlour, Cornmeal, ElbowMacaroni

Topic  9
LemonJuic, LemonPeel, LemonRind, Tahini, Dill

Topic  10
Egg, Breadcrumb, CreamOfTartar, CasterSugar, CakeFlour

Topic  11
VegetOil, CornTortilla, RussetPotato, CiderVinegar, Skinless

Topic  12
UnsaltButter, PureVanillaExtract, LightBrownSugar, PackLightBrownSugar, UnbleachFlour

Topic  13
GarlicPowder, OnionPowder, SeasonSalt, Season, DriParsley

Topic